<h1><center>Desafío Aguas Venenosas</center></h1>

Los aldeanos de la isla en la que te encuentras son víctimas de conflictos entre tribus. El método más usado para resolver conflictos ha sido el envenenamiento de aguas enemigas.

Una de las tribus está diseñando un detector de veneno. Ellos lograron identificar las 10 características mínimas y requeridas que hacen de una sustancia una sustancia venenosa para su especie (veneno.csv). Tu has recibido un dataset (sustancias_diversas.csv) de 500 sustancias diversas con las 10 características, estas sustancias fueron encontradas en distintos lagos cada una. La idea es clausurar lagos infectados. La tropa que cerrará los lagos infectados tiene sólo 50 miembros, tu debes entregar un dataset con las primeros 50 lagos que cerrará cada miembro de la tropa.
<ul>

<li>A.	Explora tus datasets.</li>
<li>B.	Explica la idea general de tu solución con dibujos, diagramas, videos, legos, bolitas de plastilina, etc.</li>
<li>C.	Implementa tu solución explicando los pasos que usas.</li>
<li>D.	Guarda las 50 sustancias en un archivo nombrado urgente_orden_de_cierre.csv</li>
<li>E.	(Opcional) Si crees que es necesario ordenarlas en algún orden específico, indícalo.</li>

<ul>

<h2>Declaración de Librerías utilizadas</h2>

In [3]:
import pandas as pd
from scipy.spatial import distance
from sklearn.preprocessing import normalize

<center><h1>A. Exploración de los Datasets</h1></center>

Carga de los datasets: El primer paso, es cargar los datos de los archivos CSV a un dataframe utilizando la librería pandas (previamente declarada), para poder manipular los datos.

Contamos con 2 archivos:
<ul>
<li>veneno.csv</li>
<li>sustancias_diversas.csv</li>
</ul>

In [4]:
#Utilizamos el método read_csv para leer los archivos y cargarlos a un dataframe cada uno
#Pasamos los parámetros header e index_col para indicar que contienen encabezado y nombres de fila.

df_veneno = pd.read_csv('./datasets/veneno.csv', header=0, index_col=0)
df_sustancias = pd.read_csv('./datasets/sustancias_diversas.csv', header=0, index_col=0)

<h2>Exploración inicial de los datasets</h2>
<p>Se utiliza el método describe, para tener una idea general del contenido de los archivos proporcionados</p>

In [7]:
print(df_sustancias.describe(percentiles=None, include=None, exclude=None))
print(df_veneno.describe(percentiles=None, include=None, exclude=None))

               v1          v2           v3          v4           v5  \
count  500.000000  500.000000   500.000000  500.000000   500.000000   
mean    20.977800   55.915400  2122.973800  165.231200  4536.858200   
std      1.667359    4.483171   165.021963   13.069945   354.197585   
min     16.800000   45.000000  1705.700000  132.100000  3624.600000   
25%     20.800000   55.400000  2111.050000  163.100000  4485.775000   
50%     21.000000   55.900000  2129.800000  164.750000  4532.750000   
75%     21.200000   56.500000  2147.625000  166.125000  4572.550000   
max     25.200000   67.200000  2556.800000  197.500000  5430.000000   

               v6          v7          v8          v9         v10  
count  500.000000  500.000000  500.000000  500.000000  500.000000  
mean     1.001600    2.107600    3.799600  538.805200   23.677000  
std      0.086559    0.170765    0.315021   41.981127    1.976893  
min      0.800000    1.700000    3.000000  430.700000   19.000000  
25%      1.000000   

<p>De la sumarización anterior podemos identificar, entre otras cosas que:</p>
<ul>
<li>El archivo veneno contiene únicamente un vector</li>
<li>La información es de tipo numérico</li>
<li>Los dataset No contienen valores vacíos o nulos</li>
<li> La desviación es proporcional al rango numérico</li>
</ul> 

<center><h2>B. Descripción de la solución</h2></center>

<img src="./img/veneno.jpg">

Se identifica que la información con la que se cuenta esta completa, por lo que no será necesario realizar grandes esfuerzos de pre-procesamiento de datos.

Definimos el objetivo de este desafío como sigue: 
<center><b>Identificar las 50 sustancias más similares al veneno</b></center>

Con el objetivo de identificar estas sustancias, se define como estrategia el Cálculo de la distancia euclidiana de las sustancias en relación al veneno.

<h2>Algoritmo</h2>

* Cargar los datasets a un dataframe
* Normalizar los datasets
* Calcular la distancia euclidiana de cada sustancia en relación al veneno
* Seleccionar las 50 sustancias con menor distancia euclidiana
* Generar el archivo: urgente_orden_de_cierre.csv

<center><h1>C. Implementación de la solución</h1></center>

In [10]:
#La carga de los datasets se hizo en un paso anterior, por lo que procedemos a realizar la normalización de los dataframes

#Normalizando los datasets (norm-l2)
veneno = df_veneno.as_matrix(columns=None)
NormL2_veneno = normalize(veneno, norm='l2')

sustancias = df_sustancias.as_matrix(columns=None)
NormL2_sustancias = normalize(sustancias, norm='l2')

#Ejemplo del dataset normalizado:
print(NormL2_sustancias)

[[ 0.00438067  0.00860172  0.43443122 ...,  0.00067395  0.10087956
   0.00352937]
 [ 0.0041515   0.01111021  0.41805621 ...,  0.00075122  0.10730643
   0.00468527]
 [ 0.0035219   0.01099143  0.3977426  ...,  0.00089015  0.08351937
   0.00454751]
 ..., 
 [ 0.00416282  0.0111938   0.42319333 ...,  0.00075688  0.10699838
   0.00474043]
 [ 0.00359713  0.01151913  0.44103284 ...,  0.00066536  0.0938788
   0.00474072]
 [ 0.00505932  0.01438389  0.48229142 ...,  0.00070331  0.14229616
   0.00487782]]


In [15]:
#Calculando la distancia euclidiana de cada sustancia vs el veneno:
distancia = [] # Instanciar lista vacía

#Recorrer el dataframe de sustancias y por cada fila, calcular la distancia euclidiana
for vector in NormL2_sustancias :
    euc_distancia = distance.euclidean(NormL2_veneno, vector)
    distancia.append(euc_distancia) #Vector resultante con las distancias

<center><h1>D y E. Generación del archivo Ordenado</h1></center>

In [16]:
#Generamos el archivo: urgente_orden_de_cierre.csv

df_sustancias['dist Euclid'] = distancia #Agregamos el vector de distancias como columna al dataset de sustancias
df_urgente = df_sustancias.nsmallest(50, 'dist Euclid', keep='first') # Seleccionamos las 50 distancias más pequeñas

df_urgente.to_csv('urgente_orden_de_cierre.csv') # Generamos el archivo.

#El resultado ORDENADO de más urgente a menos urgente es el siguiente: 
print (df_urgente)

       v1    v2      v3     v4      v5   v6   v7   v8     v9   v10  \
id                                                                   
338  20.9  56.2  2124.7  164.6  4513.6  1.0  2.1  3.8  537.2  23.8   
382  20.8  55.8  2126.4  165.6  4517.7  1.0  2.1  3.8  536.8  23.8   
135  20.9  56.4  2130.9  163.1  4524.8  1.0  2.1  3.8  536.8  23.8   
374  21.2  56.2  2131.8  164.3  4526.1  1.0  2.1  3.8  535.9  23.6   
269  21.0  56.1  2143.3  165.2  4558.3  1.0  2.1  3.8  542.2  23.8   
470  21.2  56.0  2128.5  166.1  4520.7  1.0  2.1  3.8  538.8  23.9   
370  21.0  55.5  2151.0  165.7  4573.0  1.0  2.1  3.8  541.7  23.6   
137  20.8  55.9  2113.8  164.8  4488.1  1.0  2.1  3.8  534.8  23.7   
331  21.2  56.4  2145.1  165.4  4557.8  1.0  2.1  3.8  539.0  23.8   
387  21.0  56.1  2119.0  163.5  4502.5  1.0  2.1  3.8  537.6  23.9   
130  20.8  56.2  2141.3  165.6  4555.1  1.0  2.1  3.8  540.2  23.7   
104  20.8  55.9  2128.6  164.6  4523.1  1.0  2.1  3.8  534.6  23.9   
171  20.9  55.6  214